In [1]:
!git clone https://github.com/Ci-re/youtube_shorts_scraper.git

Cloning into 'youtube_shorts_scraper'...
remote: Enumerating objects: 9, done.
remote: Counting objects: 100% (9/9), done.
remote: Compressing objects: 100% (5/5), done.
remote: Total 9 (delta 2), reused 9 (delta 2), pack-reused 0 (from 0)
Receiving objects: 100% (9/9), 253.23 KiB | 1.65 MiB/s, done.
Resolving deltas: 100% (2/2), done.


In [2]:
!pip install datasets evaluate transformers[sentencepiece]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 16.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 17.5 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [3]:
from datasets import load_from_disk
from datasets import load_dataset

trans_df = load_dataset("json", data_files="youtube_shorts_scraper/transcriptions_cleaned.json", split="train")
trans_df
# trans_df = load_from_disk("transcripts")
# trans_df

Generating train split: 0 examples [00:00, ? examples/s]

Dataset({
    features: ['label', 'transcript', 'cleaned_transcripts'],
    num_rows: 407
})

In [ ]:
trans_df.reset_format()  # Resets any previously applied format

In [ ]:
print("Columns:", trans_df.column_names)
print("First Row Example:", trans_df[:])

Columns: ['cleaned_transcripts', 'label', 'transcript']
First Row Example: {'cleaned_transcripts': ["No matter how close you are to Destiny, if you don't finish, you will stay in the same group with those who did not start. You have come too far. Are we together? You've held a  plough ready for Harvest, in Ministry, in business, refuse to be distracted. The world is full of noise makers, naysayers, people full of pain- they may not be Wicked people, just wounded and Confused people hoping they can use your pain to find meaning to their lives.", "Let me tell you the truth: the greater your love for Jesus, something happens to you. The way you see men, your relationship with Jesus, directly affects how you see men. Listen, listen, listen. The Bible says: by this shall all men know that ye are my disciples. Not when you pray in tongues, not when you fast, not when you give, he says: when you have love one towards another, you can't say: your love for Jesus is waxing hotter and hotter and 

In [ ]:
print("Current format:", trans_df.format)

Current format: {'type': None, 'format_kwargs': {}, 'columns': ['cleaned_transcripts', 'label', 'transcript'], 'output_all_columns': False}


In [ ]:
# =====================================================================

In [4]:
from huggingface_hub import notebook_login

notebook_login()

In [9]:
from datasets import load_dataset

trans_df = load_dataset("json", data_files="youtube_shorts_scraper/transcriptions_cleaned.json", split="train")
trans_df

Dataset({
    features: ['label', 'transcript', 'cleaned_transcripts'],
    num_rows: 407
})

In [10]:
trans_df = trans_df.remove_columns("transcript")
trans_df = trans_df.rename_column("cleaned_transcripts", "transcript")

In [11]:
trans_df = trans_df.train_test_split(test_size=.2)

In [13]:
trans_df["train"][0]

{'label': 230,
 'transcript': 'Anyone plotting evil, anyone plotting War, every conspiracy of dark Powers. Right now we command the elements of creation to fight everything that is Antichrist. We declare that now the Lord of Peace. By himself he will ensure that there is peace, always and by all means, in the name of Jesus Christ.'}

In [14]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilbert/distilgpt2")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/762 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [16]:
def preprocess_function(examples): return tokenizer(examples["transcript"])

In [18]:
trans_tokens = trans_df.map(
    preprocess_function,
    batched = True,
    num_proc = 2,
    remove_columns=trans_df["train"].column_names
)

Map (num_proc=2):   0%|          | 0/325 [00:00<?, ? examples/s]

Map (num_proc=2):   0%|          | 0/82 [00:00<?, ? examples/s]

In [30]:
block_size = 128

def group_texts(examples):
  concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
  total_length = len(concatenated_examples[list(examples.keys())[0]])
  print(total_length)
  if total_length >= block_size:
        total_length = (total_length // block_size) * block_size
  result = {
      k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
      for k, t in concatenated_examples.items()
  }

  result["labels"] = result["input_ids"].copy()
  return result


In [32]:
lm_dataset = trans_tokens.map(group_texts, batched=True, num_proc=4)


Map (num_proc=4):   0%|          | 0/325 [00:00<?, ? examples/s]

11873
1228513075

11582


Map (num_proc=4):   0%|          | 0/82 [00:00<?, ? examples/s]

330235023049
2947




In [35]:
from transformers import DataCollatorForLanguageModeling

tokenizer.pad_token = tokenizer.eos_token

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

In [36]:
from transformers import AutoModelForCausalLM, TrainingArguments, Trainer

model = AutoModelForCausalLM.from_pretrained("distilbert/distilgpt2")

model.safetensors:   0%|          | 0.00/353M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [40]:
training_args = TrainingArguments(
    output_dir = "transcripts-clm-model-01",
    eval_strategy = "epoch",
    learning_rate = 2e-5,
    weight_decay = 0.01,
    push_to_hub = True
)

trainer = Trainer(
    model = model,
    args = training_args,
    train_dataset = lm_dataset["train"],
    eval_dataset = lm_dataset["test"],
    data_collator = data_collator,
    tokenizer = tokenizer
)

trainer.train()

<ipython-input-40-5c0c42f6b7e8>:9: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Epoch,Training Loss,Validation Loss
1,No log,3.904141
2,No log,3.858917
3,No log,3.845065


TrainOutput(global_step=144, training_loss=3.7830768161349826, metrics={'train_runtime': 178.9255, 'train_samples_per_second': 6.355, 'train_steps_per_second': 0.805, 'total_flos': 37136800677888.0, 'train_loss': 3.7830768161349826, 'epoch': 3.0})

In [41]:
import math

eval_results = trainer.evaluate()
print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

Perplexity: 46.76


In [42]:
trainer.push_to_hub()

events.out.tfevents.1736115485.89b3b2409e61.2044.1:   0%|          | 0.00/359 [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/cirec/transcripts-clm-model-01/commit/44f2447414c57d8a232e1fce0368cdcbbfffb306', commit_message='End of training', commit_description='', oid='44f2447414c57d8a232e1fce0368cdcbbfffb306', pr_url=None, repo_url=RepoUrl('https://huggingface.co/cirec/transcripts-clm-model-01', endpoint='https://huggingface.co', repo_type='model', repo_id='cirec/transcripts-clm-model-01'), pr_revision=None, pr_num=None)

In [49]:
prompt = "Without God"

from transformers import pipeline

generator = pipeline("text-generation", model="cirec/transcripts-clm-model-01")
generator(prompt)

Device set to use cuda:0


[{'generated_text': 'Without God has mercy upon our children. We have mercy upon the children. They cannot forgive themselves. They cannot forgive themselves. They see a son, that is called the Savior; he is called the "true savior." He only has that ability to'}]